In [1]:
import pandas as pd
import re
import numpy as np
import regex

In [2]:
data = pd.read_csv('../ete3/Plt_sci_publications_geo_species_8.26.csv', low_memory=False)

# Get dict of names and genders to reduce calls to GenderAPI

In [3]:
names_dict = data[['First_names_currated', 'inferred_gender']].dropna()

In [4]:
names_dict['First_names_currated'] = names_dict['First_names_currated'].str.strip()

In [5]:
# need to drop duplicates here otherwise issues when merging
names_dict = names_dict.drop_duplicates(subset=['First_names_currated'])

In [6]:
names_dict = names_dict.rename(columns = {'First_names_currated': 'First name'})

In [7]:
diff_loc_enriched = pd.read_csv('diff_location_enriched.csv')

In [8]:
one_loc_enriched = pd.read_csv('one_location_enriched.csv')

In [9]:
names_dict_diff = diff_loc_enriched[['First name', 'ga_gender']]
names_dict_diff = names_dict_diff.rename(columns = {'ga_gender': 'inferred_gender'})

In [10]:
names_dict_one = one_loc_enriched[['First name', 'ga_gender']]
names_dict_one = names_dict_one.rename(columns = {'ga_gender': 'inferred_gender'})

In [11]:
names_dict_final = pd.concat([names_dict, names_dict_diff, names_dict_one])

In [334]:
names_dict_final = names_dict_final.drop_duplicates(subset='First name')

# Fix problems

Running the line
    
    re.findall(one_loc_out.iloc[i]['Corresponding_author_last_name']+"+[^;]*", one_loc_out.iloc[i]['Author Full Names'])
    
in case 2.1 made me realize entries 24144, 25071, and 40426 have an extra semicolon in the corresp. author addresses. Fix these manually here.

In [12]:
for i in [24144, 25071, 40426]:
    data.at[i,'Corresponding author Addresses'] = data.loc[i]['Corresponding author Addresses'].replace(';', '')

# Get first names

In [13]:
# if they have no names we don't care
data = data.dropna(subset='Author Full Names')

In [14]:
# gather all names of the form Lastname, AB or Lastname, A.B.
all_initials = []

for i in data.index:
    author_list = data.loc[i]['Author Full Names'].split('; ')
    if len([x for x in author_list if not re.findall(', [A-Z]{1,}$',x) and not re.findall(', [A-Z]. [A-Z].$',x) and not re.findall(', [A-Z].$',x)]) == 0:
        all_initials.append(i)

# remove entries with names found above from the dataset
data = data[~data.index.isin(all_initials)]

In [15]:
# many of these are one author with multiple affiliations

mult_corresp = []

for i in range(len(data['Corresponding author Addresses'])):
    if ';' in data.iloc[i]['Corresponding author Addresses']:
        mult_corresp.append(i)

## Case 1: one author, mult affiliations
Don't need to do anything with these

In [16]:
one_author = []

for i in mult_corresp:
    test_str = data.iloc[i]['Corresponding author Addresses']
    test_str_list = test_str.split(';')
    stripped_list = [re.findall('[^()]+', i)[0].strip() for i in test_str_list]
    
    if len(set(stripped_list)) == 1:
        one_author.append(i)

## Case 2: mult authors

In [17]:
# get only true mult authors
mult_authors = sorted(list(set(mult_corresp)-set(one_author)))

In [18]:
mult_authors_df = data.iloc[mult_authors]

In [19]:
mult_authors_df['Corresponding author Addresses'] = mult_authors_df['Corresponding author Addresses'].str.split(';')

/tmp/ipykernel_451133/4070540700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_authors_df['Corresponding author Addresses'] = mult_authors_df['Corresponding author Addresses'].str.split(';')


In [20]:
mult_authors_df['Corresponding author geocoords'] = mult_authors_df['Corresponding author geocoords'].str.split(';')

/tmp/ipykernel_451133/1495248539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_authors_df['Corresponding author geocoords'] = mult_authors_df['Corresponding author geocoords'].str.split(';')


### Subcase 1: Authors all have same location
In this case, there will be only one location listed under corresp author geocoords

In [110]:
one_loc_df = mult_authors_df[mult_authors_df['Corresponding author geocoords'].map(len)==1]

In [111]:
one_loc_df['map_props'] = [1 for i in range(len(one_loc_df))]

/tmp/ipykernel_451133/2763481392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_loc_df['map_props'] = [1 for i in range(len(one_loc_df))]


In [112]:
one_loc_out = one_loc_df.explode('Corresponding author Addresses')
one_loc_out['map_props'] /= one_loc_out['map_props'].groupby(level=0).transform('count')

In [113]:
one_loc_out['Corresponding_author_last_name'] = one_loc_out['Corresponding author Addresses'].str.split(', ').str[0]

In [114]:
one_loc_out['Corresponding_author_last_name'] = one_loc_out['Corresponding_author_last_name'].str.strip()

In [115]:
# names that just have initials as first names will just have the initials as the first name
one_loc_out["first and last names"]=[re.findall(one_loc_out.iloc[i]['Corresponding_author_last_name']+"+[^;]*", one_loc_out.iloc[i]['Author Full Names']) for i in range(len(one_loc_out))]

In [116]:
one_loc_out = one_loc_out.reset_index()

In [117]:
no_authors = []

for i in one_loc_out.index:
    if len(one_loc_out.loc[i]['first and last names']) == 0:
        no_authors.append(i)
        
no_authors_df = one_loc_out.loc[no_authors]

In [118]:
# need to do the german last name thing
new_last_names = []

for i in list(no_authors_df.index):
    old_last_name = no_authors_df.loc[i]['Corresponding_author_last_name']
    author_full_name = no_authors_df.loc[i]['Author Full Names']
    # if the old last name is one insertion away from something in the author full names, then consider it a match
    new_last_name = regex.findall("("+old_last_name+"){i<=1,d<=1}", author_full_name, overlapped=True)
    # if there are still no matches, ignore for now
    if len(new_last_name) == 0:
        pass    
    else:
        new_last_names.append([i,new_last_name[0]])

In [119]:
# replace last names in old dataframe with new last names (which contain an extra letter)
for i in range(len(new_last_names)):
    no_authors_df.at[new_last_names[i][0],'Corresponding_author_last_name'] = new_last_names[i][1]

In [120]:
no_authors_df["first and last names"]=[re.findall(no_authors_df.loc[i]['Corresponding_author_last_name']+"+[^;]*", no_authors_df.loc[i]['Author Full Names']) for i in no_authors_df.index]

In [121]:
for i in no_authors_df.index:
    one_loc_out.at[i, 'first and last names'] = no_authors_df.at[i, 'first and last names']

In [122]:
one_loc_out[one_loc_out["first and last names"].map(len) != 1]["first and last names"]

11                     [Liu, Wen, Liu, Tianzhong]
54                 [Lin, Li, Wen-Dong, Liu, Ying]
78                            [Yu, Ying, Yu-Huan]
80                            [Yu, Ying, Yu-Huan]
100                      [Yunxiao, Yuan, Yu, Fei]
                           ...                   
27014                 [Chen Zhi-Wei, Chen Ru-Kai]
27024                [Zhou, Yan, Zhou, Changyong]
27026                [Zhou, Yan, Zhou, Changyong]
27037    [Zambolim, Eunize M., Zambolim, Laercio]
27039                       [Lin, Yi-Hua, Li, Yu]
Name: first and last names, Length: 5897, dtype: object

In [123]:
# this is to find authors that have the same last name as other authors
mult_author_df = one_loc_out[one_loc_out["first and last names"].map(len) != 1]

In [124]:
mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding author Addresses'].str.findall(r'^[^,]*,..').str[0]

/tmp/ipykernel_451133/3790255992.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding author Addresses'].str.findall(r'^[^,]*,..').str[0]


In [125]:
mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding_author_last_name'].str.strip()

/tmp/ipykernel_451133/4227390828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding_author_last_name'].str.strip()


In [126]:
# use previous cell to find first and last names of corresponding authors
author_matches = []

for i in mult_author_df.index:
    mylist = mult_author_df.loc[i]['first and last names']
    r = re.compile(str(mult_author_df.loc[i]['Corresponding_author_last_name'])+".*")
    newlist = list(filter(r.match, mylist))
    author_matches.append(newlist)
    
mult_author_df['first and last names'] = author_matches

/tmp/ipykernel_451133/1175752697.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_author_df['first and last names'] = author_matches


In [129]:
for i in mult_author_df.index:
    one_loc_out.at[i,'first and last names'] = mult_author_df.at[i,'first and last names']

In [133]:
one_loc_out = one_loc_out[one_loc_out["first and last names"].map(len)==1]

In [134]:
one_loc_out["first and last names"] = [i[0].split(', ') for i in one_loc_out["first and last names"]]

In [135]:
one_loc_out["First name"] = [i[-1] for i in one_loc_out["first and last names"]]

In [137]:
one_loc_names["Last name"] = [i[0] for i in one_loc_names["first and last names"]]

In [249]:
one_loc_names = one_loc_out.merge(names_dict_final, how='left', on='First name')
one_loc_names = one_loc_names.loc[one_loc_names.astype(str).drop_duplicates().index]

In [250]:
one_loc_names = one_loc_names.sort_values(by=['index']).reset_index(drop=True)

### Subcase 2: All authors have different locations

In [264]:
diff_loc_df = mult_authors_df[mult_authors_df['Corresponding author geocoords'].map(len)==mult_authors_df['Corresponding author Addresses'].map(len)]

In [265]:
diff_loc_df['map_props'] = [1 for i in range(len(diff_loc_df))]

/tmp/ipykernel_451133/720741673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_loc_df['map_props'] = [1 for i in range(len(diff_loc_df))]


In [266]:
diff_loc_out = diff_loc_df.explode(['Corresponding author Addresses','Corresponding author geocoords'])
diff_loc_out['map_props'] /= diff_loc_out['map_props'].groupby(level=0).transform('count')

In [275]:
diff_loc_out['Corresponding_author_last_name'] = diff_loc_out['Corresponding author Addresses'].str.split(', ').str[0]

In [276]:
diff_loc_out['Corresponding_author_last_name'] = diff_loc_out['Corresponding_author_last_name'].str.strip()

In [277]:
# names that just have initials as first names will just have the initials as the first name
diff_loc_out["first and last names"]=[re.findall(diff_loc_out.iloc[i]['Corresponding_author_last_name']+"+[^;]*", diff_loc_out.iloc[i]['Author Full Names']) for i in range(len(diff_loc_out))]

In [280]:
diff_loc_out = diff_loc_out.reset_index()

In [281]:
# find entries that have no corresponding author name extracted
no_authors_diff = []

for i in diff_loc_out.index:
    if len(diff_loc_out.loc[i]['first and last names']) == 0:
        no_authors_diff.append(i)
        
no_authors_diff_df = diff_loc_out.loc[no_authors_diff]

In [282]:
# need to do the german last name thing
new_last_names_diff = []

for i in list(no_authors_diff_df.index):
    old_last_name = no_authors_diff_df.loc[i]['Corresponding_author_last_name']
    author_full_name = no_authors_diff_df.loc[i]['Author Full Names']
    # if the old last name is one insertion away from something in the author full names, then consider it a match
    new_last_name = regex.findall("("+old_last_name+"){i<=1,d<=1}", author_full_name, overlapped=True)
    # if there are still no matches, ignore for now
    if len(new_last_name) == 0:
        pass    
    else:
        new_last_names_diff.append([i,new_last_name[0]])

In [283]:
# replace last names in old dataframe with new last names (which contain an extra letter)
for i in range(len(new_last_names_diff)):
    no_authors_diff_df.at[new_last_names_diff[i][0],'Corresponding_author_last_name'] = new_last_names_diff[i][1]

In [284]:
no_authors_diff_df["first and last names"]=[re.findall(no_authors_diff_df.loc[i]['Corresponding_author_last_name']+"+[^;]*", no_authors_diff_df.loc[i]['Author Full Names']) for i in no_authors_diff_df.index]

In [285]:
for i in no_authors_diff_df.index:
    diff_loc_out.at[i, 'first and last names'] = no_authors_diff_df.at[i, 'first and last names']

In [ ]:
# this is the problem

In [ ]:
# drop people with no first names at this point
# diff_loc_out = diff_loc_out[diff_loc_out["first and last names"].map(len)==1]

In [292]:
# this is to find authors that have the same last name as other authors
mult_author_df = diff_loc_out[diff_loc_out["first and last names"].map(len) != 1]

In [293]:
mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding author Addresses'].str.findall(r'^[^,]*,..').str[0]

/tmp/ipykernel_451133/3790255992.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding author Addresses'].str.findall(r'^[^,]*,..').str[0]


In [294]:
mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding_author_last_name'].str.strip()

/tmp/ipykernel_451133/4227390828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_author_df['Corresponding_author_last_name'] = mult_author_df['Corresponding_author_last_name'].str.strip()


In [295]:
# use previous cell to find first and last names of corresponding authors
author_matches = []

for i in mult_author_df.index:
    mylist = mult_author_df.loc[i]['first and last names']
    r = re.compile(str(mult_author_df.loc[i]['Corresponding_author_last_name'])+".*")
    newlist = list(filter(r.match, mylist))
    author_matches.append(newlist)
    
mult_author_df['first and last names'] = author_matches

/tmp/ipykernel_451133/1175752697.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_author_df['first and last names'] = author_matches


In [296]:
for i in mult_author_df.index:
    diff_loc_out.at[i,'first and last names'] = mult_author_df.at[i,'first and last names']

In [298]:
# drop names that can't be matched at this point
diff_loc_out = diff_loc_out[diff_loc_out["first and last names"].map(len) == 1]

In [301]:
diff_loc_out["first and last names"] = [i[0].split(', ') for i in diff_loc_out["first and last names"]]

/tmp/ipykernel_451133/1656389199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_loc_out["first and last names"] = [i[0].split(', ') for i in diff_loc_out["first and last names"]]


In [302]:
diff_loc_out["First name"] = [i[-1] for i in diff_loc_out["first and last names"]]

/tmp/ipykernel_451133/642284433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_loc_out["First name"] = [i[-1] for i in diff_loc_out["first and last names"]]


In [303]:
diff_loc_out["Last name"] = [i[0] for i in diff_loc_out["first and last names"]]

/tmp/ipykernel_451133/3864045261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diff_loc_out["Last name"] = [i[0] for i in diff_loc_out["first and last names"]]


In [337]:
diff_loc_names = diff_loc_out.merge(names_dict_final, on='First name', how='left')
# diff_loc_names = diff_loc_names.loc[diff_loc_names.astype(str).drop_duplicates().index]

In [339]:
diff_loc_names = diff_loc_names.set_index('index')

# Merge back with original data

In [341]:
data = pd.read_csv('../ete3/Plt_sci_publications_geo_species_8.26.csv', low_memory=False)

In [342]:
one_loc_names_merged = (one_loc_names.groupby(['index'])
    .agg({'inferred_gender_y': lambda x: x.tolist(), 'first and last names': lambda x: x.tolist()}))

In [343]:
diff_loc_names_merged = (diff_loc_names.groupby(['index'])
    .agg({'inferred_gender_y': lambda x: x.tolist(), 'first and last names': lambda x: x.tolist()}))

In [344]:
for i in one_loc_names_merged.index:
    data.at[i,'inferred_gender'] = one_loc_names_merged.at[i,'inferred_gender_y']
    data.at[i,'first and last names'] = one_loc_names_merged.at[i,'first and last names']

In [345]:
for i in diff_loc_names_merged.index:
    data.at[i,'inferred_gender'] = diff_loc_names_merged.at[i,'inferred_gender_y']
    data.at[i,'first and last names'] = diff_loc_names_merged.at[i,'first and last names']

In [255]:
# data.to_csv('data_full_gender.csv', index = False)

In [256]:
# data.to_pickle('data_full_gender.pkl')

In [178]:
diff_loc_out[diff_loc_out["first and last names"].map(len)==diff_loc_out["first and last names"].map(len)]

,index,Author Full Names,Article Title,Journal,impact factor (2020),Author Keywords,Keywords Plus,Abstract,Addresses--all authors,Nations--all authors,...,Locations--all authors,Geocoords--all authors,Corresponding author locations,Corresponding author geocoords,Number of authoring countries,Number of corresponding countries,unique_NCBI,map_props,First name,Last name
0,15,"Amaral, Raquel; Sevcikova, Tereza; Elias, Mare...",CharaciopsisBorzi belongs to the Eustigmatophy...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Characiopsis; Eustigmataceae group; Eustigmato...,ALGAL CLASS; PHYLOGENY; ULTRASTRUCTURE; NOMENC...,"Characiopsis, established by Borzi in 1895, is...","[Amaral, Raquel; Santos, Lilia M. A.] Univ Coi...",Portugal; Czechia,...,"Coimbra, Portugal; Ostrava, Czechia","(40.20331450000001, -8.4102573); (49.8209226, ...","Coimbra, Portugal; Ostrava, Czechia","(40.20331450000001, -8.4102573)",2,2,"[425071, 2713033, 1431827, 5747, 44429, 271303...",0.5,Raquel,Amaral
1,15,"Amaral, Raquel; Sevcikova, Tereza; Elias, Mare...",CharaciopsisBorzi belongs to the Eustigmatophy...,EUROPEAN JOURNAL OF PHYCOLOGY,0.874,Characiopsis; Eustigmataceae group; Eustigmato...,ALGAL CLASS; PHYLOGENY; ULTRASTRUCTURE; NOMENC...,"Characiopsis, established by Borzi in 1895, is...","[Amaral, Raquel; Santos, Lilia M. A.] Univ Coi...",Portugal; Czechia,...,"Coimbra, Portugal; Ostrava, Czechia","(40.20331450000001, -8.4102573); (49.8209226, ...","Coimbra, Portugal; Ostrava, Czechia","(49.8209226, 18.262524300000006)",2,2,"[425071, 2713033, 1431827, 5747, 44429, 271303...",0.5,Marek,Elias
2,50,"Bernard, Miriam S.; Strittmatter, Martina; Mur...","Diversity, biogeography and host specificity o...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,5'COI; barcoding; biogeography; endophytes; ho...,ALGA CAULERPA-TAXIFOLIA; NUCLEAR RIBOSOMAL DNA...,Endophytic filamentous brown algae are known t...,"[Bernard, Miriam S.; Heesch, Svenja; Leblanc, ...",France; Scotland; South Korea,...,"Roscoff, France; Oban, Argyll, Scotland; Banch...","(37.41379999999999, 127.5183); (48.703091, -4....","Roscoff, France; Santec, France","(48.703091, -4.0292009)",3,1,"[90893, 64910, 64922, 64929, 309358, 172714, 6...",0.5,Miriam S.,Bernard
3,50,"Bernard, Miriam S.; Strittmatter, Martina; Mur...","Diversity, biogeography and host specificity o...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,5'COI; barcoding; biogeography; endophytes; ho...,ALGA CAULERPA-TAXIFOLIA; NUCLEAR RIBOSOMAL DNA...,Endophytic filamentous brown algae are known t...,"[Bernard, Miriam S.; Heesch, Svenja; Leblanc, ...",France; Scotland; South Korea,...,"Roscoff, France; Oban, Argyll, Scotland; Banch...","(37.41379999999999, 127.5183); (48.703091, -4....","Roscoff, France; Santec, France","(48.726199, -3.985325)",3,1,"[90893, 64910, 64922, 64929, 309358, 172714, 6...",0.5,Akira F.,Peters
4,132,"D'Archino, Roberta; Lin, Showe-Mei; Gabrielson...","Why one species in New Zealand, Pugetia delica...",EUROPEAN JOURNAL OF PHYCOLOGY,0.874,DNA sequencing; endemic genera; large-subunit ...,SUBUNIT RDNA; GIGARTINACEAE RHODOPHYTA; MAXIMU...,"Blade-forming red algae occur worldwide and, p...","[D'Archino, Roberta] Natl Inst Water & Atmosph...",New Zealand; Taiwan; United States,...,"Wellington, New Zealand; Keelung, Taiwan; Chap...","(-41.2923814, 174.77874630000005); (25.1275997...","Wellington, New Zealand; Keelung, Taiwan","(-41.2923814, 174.77874630000005)",3,2,"[217477, 1789642, 1789644, 1789643, 31430]",0.5,Roberta,D'Archino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10076,290266,"Noorizadeh, Sina; Khakvar, Reza; Golmohammadi,...",Multilocus genotyping of 'Candidatus Phytoplas...,TROPICAL PLANT PATHOLOGY,1.512,Genetic diversity; Group II intron; Phylogeny,ASTER YELLOWS; PROTEIN; TAXON; LIME; CLASSIFIC...,"Witches' broom disease of lime (WBDL), associa...","[Noorizadeh, Sina; Khakvar, Reza] Tabriz Univ,...",Iran,...,"Tabriz, Iran; Ramsar, Iran; Zarghan, Iran","(29.7641623, 52.7134927); (36.9268274, 50.6430.

### Subcase 3: Some authors have different locations. Drop these
Some authors may correspond to more than one location and or/ multiple authors may correspond to one location

In [346]:
# get everything not in subcases 1 or 2
step1 = mult_authors_df[~mult_authors_df.isin(one_loc_df)].dropna(subset=['Author Full Names'])
mult_loc_df = step1[~step1.isin(diff_loc_df)].dropna(subset=['Author Full Names'])

In [347]:
data = data[~data.index.isin(mult_loc_df.index)]

In [348]:
data.to_pickle('data_full_gender.pkl')